In [47]:
import torch
import numpy as np
import random
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "2,3"  # Use the 3rd and 4th GPU. Indexing starts from 0.
#os.environ["CUDA_LAUNCH_BLOCKING"] = "1"  # Use the 3rd and 4th GPU. Indexing starts from 0.

def set_seed(seed_value=42):
    """Set seed for reproducibility."""
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)

    # When using GPU
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        # torch.backends.cudnn.benchmark = True
        
set_seed(3047)

In [ ]:
# create customdataset that loads in all images for a particular volume, particular series_id
# resize/crop the images to 224x224 and shrink/increase them to 15 slices. 
# crop by reducing the width, so it conforms more like a 1.15 ratio between width and height
# Do it for RGB channels

# Afterwards assign them labels based on their injury status

In [50]:
import os
import pandas as pd
import numpy as np
import torch
def collect_image_paths(base_folder):
    """
    Traverse the directory structure under base_folder and collect paths to all image files.
    
    Args:
    - base_folder (str): The main directory containing all series_id subfolders.
    
    Returns:
    - DataFrame: Contains columns 'series_id', 'organ', 'image_path', and 'image_name'.
    """
    data = []

    # Iterate through each series_id in the main folder
    for series_id in os.listdir(base_folder):
        series_path = os.path.join(base_folder, series_id)
        
        # Check if it's a directory and not a file
        if os.path.isdir(series_path):
            
            # Iterate through each organ folder under the current series_id
            for organ in ['bowel', 'kidneys', 'liver', 'spleen']:
                organ_path = os.path.join(series_path, organ)
                
                # Collect all .jpg image paths under the organ directory
                organ_images = [os.path.join(organ_path, fname) for fname in os.listdir(organ_path) if fname.endswith('.jpg')]
                
                for image_path in organ_images:
                    image_name = os.path.basename(image_path).replace('.jpg', '')  # Extract filename without extension
                    data.append({
                        'series_id': series_id,
                        'organ': organ,
                        'image_path': image_path,
                        'image_name': image_name
                    })
                    
    df = pd.DataFrame(data)
    df['series_id'] = df['series_id'].astype(int)
    df['image_name'] = df['image_name'].astype(int)
    return df

# Example usage:
base_folder = 'series_image_split'
all_image_paths = collect_image_paths(base_folder)
all_image_paths = all_image_paths.sort_values(by=['series_id','image_name'],axis=0)

In [51]:
# Load data
train_info = pd.read_csv('train.csv')
train_info.patient_id = train_info.patient_id.astype(str)

mapping_df = pd.read_csv('patient_series_mapping.csv')
mapping_df = mapping_df.drop('Unnamed: 0',axis=1)
mapping_df = mapping_df[['patient_id','series_id']]
mapping_df['patient_id'] = mapping_df['patient_id'].astype(int)
mapping_df['series_id'] = mapping_df['series_id'].astype(int)

merged_df = all_image_paths.merge(mapping_df[['series_id', 'patient_id']], on='series_id', how='left')

y_original_format = train_info.drop(['bowel_healthy','extravasation_healthy','any_injury'], axis=1)
y_original_format['patient_id'] = y_original_format['patient_id'].astype(int)

merged_df = merged_df.merge(y_original_format[['patient_id','bowel_injury', 'extravasation_injury', 'kidney_healthy',
       'kidney_low', 'kidney_high', 'liver_healthy', 'liver_low', 'liver_high',
       'spleen_healthy', 'spleen_low', 'spleen_high']], on='patient_id', how='left')

y_original_format = train_info.drop(['patient_id','bowel_healthy','extravasation_healthy','any_injury'], axis=1).values

In [52]:
import cv2
import torch
from torch.utils.data import Dataset

image_size = 320

class OrganTrainDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform
        self.series_ids = self.dataframe['series_id'].unique()
        # self.organs = ['liver', 'spleen', 'kidneys']
        self.organs = ['liver', 'spleen', 'kidneys', 'bowel']
        
        # Define columns that contain labels
        self.label_columns = ['bowel_injury', 'extravasation_injury', 'kidney_healthy', 
                              'kidney_low', 'kidney_high', 'liver_healthy', 'liver_low', 
                              'liver_high', 'spleen_healthy', 'spleen_low', 'spleen_high']

    def __len__(self):
        return len(self.series_ids)

    def __getitem__(self, idx):
        series_id = self.series_ids[idx]
        curr_df = self.dataframe[self.dataframe['series_id'] == series_id]

        # 4 organs * 15 depth 
        images = []
        for org in self.organs:
            org_df = curr_df[curr_df.organ == org]
            org_path = np.array(org_df.image_path)
            if len(org_path)==0:
                print(series_id)
                print('Something wrong')
            else:
                org_images = []  # Store images for the current organ
                for path in org_path:
                    image = cv2.imread(path)
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                    if self.transform:
                        image = self.transform(image=image)['image']
                    image = image.transpose(2, 0, 1).astype(np.float32) / 255.
                    org_images.append(image)
                
                # Stack images for the current organ and resize depth to 15
                org_images = np.stack(org_images, 1)
                org_images = torch.tensor(org_images)
                resized_org_images = F.interpolate(org_images.unsqueeze(0), size=(10, image_size, image_size), mode='trilinear', align_corners=True).squeeze(0)
                # print(resized_org_images.shape)
                images.append(resized_org_images)
    
        # Stack all organs' images together
        images = torch.cat(images, 1)
        images = images.transpose(1, 0)
        
        # print(images.shape)
        
        # Extract labels for the given series_id
        labels = curr_df.iloc[0][self.label_columns].values.astype(np.float32)
        
        return images, torch.tensor(labels)

In [53]:
import albumentations
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
import torch.cuda.amp as amp
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

import timm

In [54]:
transforms_train = albumentations.Compose([
    albumentations.RandomBrightnessContrast(contrast_limit=0.2, brightness_limit=0, p=1.0),
    albumentations.ShiftScaleRotate(shift_limit=0.2, scale_limit=0.2, 
                                    rotate_limit=20, border_mode=cv2.BORDER_CONSTANT, p=1.0),
    albumentations.CoarseDropout(max_holes=2, max_height=int(0.3*image_size), 
                          max_width=int(0.3*image_size), fill_value=0, always_apply=True, p=1.0),
])


transforms_valid = albumentations.Compose([
    albumentations.Resize(image_size, image_size),
])

In [55]:
n_organs = 4  # Number of organs
n_slice_per_c = 10
bs = 1
out_dim = 5
pretrained = True
drop_path_rate = 0
drop_rate_last = 0.3
drop_rate = 0
backbone = 'convnext_nano'
use_amp = True

in_chans = 3

class TimmModelType2(nn.Module):
    def __init__(self, backbone, pretrained=False):
        super(TimmModelType2, self).__init__()

        self.encoder = timm.create_model(
            backbone,
            in_chans=in_chans,
            num_classes=out_dim,
            features_only=False,
            drop_rate=drop_rate,
            drop_path_rate=drop_path_rate,
            pretrained=pretrained
        )

        if 'efficient' in backbone:
            hdim = self.encoder.conv_head.out_channels
            self.encoder.classifier = nn.Identity()
        elif 'convnext' in backbone:
            hdim = self.encoder.head.fc.in_features
            self.encoder.head.fc = nn.Identity()

        self.lstm = nn.LSTM(hdim, 256, num_layers=2, dropout=drop_rate, bidirectional=True, batch_first=True)
        self.head = nn.Sequential(
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.Dropout(drop_rate_last),
            nn.LeakyReLU(0.1),
            nn.Linear(256, out_dim),
        )
        self.lstm2 = nn.LSTM(hdim, 256, num_layers=2, dropout=drop_rate, bidirectional=True, batch_first=True)
        self.head2 = nn.Sequential(
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.Dropout(drop_rate_last),
            nn.LeakyReLU(0.1),
            nn.Linear(256, 1),
        )
    
    def forward(self, x):  # (bs, n_slice_per_c * n_organs * in_chans, H, W)
        bs = x.shape[0]
        x = x.view(bs * n_slice_per_c * n_organs, in_chans, image_size, image_size)
        feat = self.encoder(x)
        feat = feat.view(bs, n_slice_per_c * n_organs, -1)
        feat1, _ = self.lstm(feat)
        feat1 = feat1.contiguous().view(bs * n_slice_per_c * n_organs, 512)
        feat2, _ = self.lstm2(feat)
        return self.head(feat1), self.head2(feat2[:, 0])

class TimmModelWithDualLSTM(nn.Module):
    def __init__(self, backbone, pretrained=False):
        super(TimmModelWithDualLSTM, self).__init__()

        self.encoder = timm.create_model(
            backbone,
            in_chans=in_chans,
            num_classes=1,
            features_only=False,
            drop_rate=drop_rate,
            drop_path_rate=drop_path_rate,
            pretrained=pretrained
        )

        # Determine the dimension of the features from the encoder
        if 'efficient' in backbone:
            hdim = self.encoder.conv_head.out_channels
            self.encoder.classifier = nn.Identity()
        elif 'convnext' in backbone:
            hdim = self.encoder.head.fc.in_features
            self.encoder.head.fc = nn.Identity()

        # Binary classification LSTM
        self.lstm = nn.LSTM(hdim, 256, num_layers=1, dropout=drop_rate, bidirectional=True, batch_first=True)
        self.head = nn.Sequential(
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.Dropout(drop_rate_last),
            nn.LeakyReLU(0.1),
            nn.Linear(256, 2)  # 2 binary labels
        )

        # Multiclass classification LSTM
        self.lstm2 = nn.LSTM(hdim, 256, num_layers=1, dropout=drop_rate, bidirectional=True, batch_first=True)
        self.head2 = nn.Sequential(
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.Dropout(drop_rate_last),
            nn.LeakyReLU(0.1),
            nn.Linear(256, 9)  # 9 multiclass labels
        )

    def forward(self, x):  # (bs, n_slice_per_c * n_organs * in_chans, H, W)
        bs = x.shape[0]
        x = x.view(bs * n_slice_per_c * n_organs, in_chans, image_size, image_size)
        feat = self.encoder(x)
        feat = feat.view(bs, n_slice_per_c * n_organs, -1)
        
        feat1, _ = self.lstm(feat)
        feat1 = torch.mean(feat1, dim=1)
        #feat1 = feat1.contiguous().view(bs * n_slice_per_c * n_organs, 512)
        binary_out = self.head(feat1)
              
        feat2, _ = self.lstm2(feat)
        feat2 = torch.mean(feat2, dim=1)
        #feat2 = feat2.contiguous().view(bs * n_slice_per_c * n_organs, 512)
        multiclass_out = self.head2(feat2)
        
        return binary_out, multiclass_out#[0,:], multiclass_out[0,:]


In [58]:
def mixup(input, truth, clip=[0, 1]):
    indices = torch.randperm(input.size(0))
    shuffled_input = input[indices]
    
    # Assuming truth is in the shape (batch_size, n_labels)
    # where n_labels includes binary and multiclass labels
    shuffled_labels = truth[indices]
    
    lam = np.random.uniform(clip[0], clip[1])
    mixed_input = input * lam + shuffled_input * (1 - lam)
    
    # Mix labels
    mixed_labels = truth * lam + shuffled_labels * (1 - lam)
    
    return mixed_input, mixed_labels, shuffled_labels, lam

def binary_criterion(logits, targets, model_device):
    # Split logits
    bowel_logits = logits[:, 0]
    extravasation_logits = logits[:, 1]

    # Split targets
    bowel_targets = targets[:, 0]
    extravasation_targets = targets[:, 1]
    
    # Binary Loss for bowel and extravasation
    bowel_losses = nn.BCEWithLogitsLoss(reduction='none')(bowel_logits, bowel_targets)
    bowel_losses[bowel_targets > 0] *= 2.
    
    extravasation_losses = nn.BCEWithLogitsLoss(reduction='none')(extravasation_logits, extravasation_targets)
    extravasation_losses[extravasation_targets > 0] *= 2.

    # Combine and normalize the binary losses
    all_losses = torch.cat([bowel_losses, extravasation_losses], dim=0)
    norm = torch.ones(all_losses.shape[0]).to(model_device)
    norm[torch.cat([bowel_targets, extravasation_targets], dim=0) > 0] *= 2
    total_loss = all_losses.sum() / norm.sum()

    return total_loss

def extravasation_binary_criterion(logits, targets, model_device):
    # Split logits
    extravasation_logits = logits[:, 1]

    # Split targets
    extravasation_targets = targets[:, 1]
    
    # Binary Loss for bowel and extravasation  
    extravasation_losses = nn.BCEWithLogitsLoss(reduction='none')(extravasation_logits, extravasation_targets)

    # Combine and normalize the binary losses
    all_losses = extravasation_losses
    norm = torch.ones(all_losses.shape[0]).to(model_device)
    norm[extravasation_losses > 0] *= 4
    total_loss = all_losses.sum() / norm.sum()

    return total_loss
    
def bowel_binary_criterion(logits, targets, model_device):
    # Split logits
    bowel_logits = logits[:, 0]

    # Split targets
    bowel_targets = targets[:, 0]
    
    # Binary Loss for bowel and extravasation
    bowel_losses = nn.BCEWithLogitsLoss(reduction='none')(bowel_logits, bowel_targets)
    
    # Combine and normalize the binary losses
    all_losses = bowel_losses
    norm = torch.ones(all_losses.shape[0]).to(model_device)
    norm[bowel_targets > 0] *= 4
    total_loss = all_losses.sum() / norm.sum()

    return total_loss

def compute_organ_criterion(logits, targets, slice_indices, model_device):
    organ_logits = logits[:, slice_indices]
    organ_targets = targets[:, slice_indices]

    # Multiclass Loss
    #ce = nn.CrossEntropyLoss(reduction='none', weight = class_weights)
    ce = nn.CrossEntropyLoss(reduction='none')

    organ_losses = ce(organ_logits, organ_targets)

    # Normalization
    norm = torch.ones(organ_losses.shape[0]).to(model_device)
    organ_targets_flat = organ_targets.argmax(dim=1)
    norm[organ_targets_flat > 0] *= 4
    
    # Calculate the final loss
    total_loss = organ_losses.sum() / norm.sum()

    return total_loss

def kidney_multi_criterion(logits, targets, model_device):
    return compute_organ_criterion(logits, targets, slice(0,3), model_device)

def liver_multi_criterion(logits, targets, model_device):
    return compute_organ_criterion(logits, targets, slice(3,6), model_device)

def spleen_multi_criterion(logits, targets, model_device):
    return compute_organ_criterion(logits, targets, slice(6,9), model_device)
    
def multiclass_criterion(logits, targets, model_device):
    # Split logits
    print(logits.shape)
    
    kidney_logits = logits[:, 0:3]  # Classes 0, 1, 2
    liver_logits = logits[:, 3:6]   # Classes 3, 4, 5
    spleen_logits = logits[:, 6:9]  # Classes 6, 7, 8

    # Split targets
    kidney_targets = targets[:, 0:3]
    liver_targets = targets[:, 3:6]
    spleen_targets = targets[:, 6:9]

    # Multiclass Losses
    ce = nn.CrossEntropyLoss(reduction='none')
    kidney_losses = ce(kidney_logits, kidney_targets)
    liver_losses = ce(liver_logits, liver_targets)
    spleen_losses = ce(spleen_logits, spleen_targets)

    # Combine and normalize the multiclass losses
    all_losses = torch.cat([kidney_losses, liver_losses, spleen_losses], dim=0)
    norm = torch.ones(all_losses.shape[0]*3).to(model_device)

    # Reshape each organ's targets
    kidney_targets_flat = kidney_targets.reshape(-1)
    liver_targets_flat = liver_targets.reshape(-1)
    spleen_targets_flat = spleen_targets.reshape(-1)
    
    # Concatenate the reshaped targets
    combined_targets = torch.cat([kidney_targets_flat, liver_targets_flat, spleen_targets_flat])
    
    # Update the norm based on the targets
    norm[combined_targets > 0] *= 2
    
    # Calculate the final loss
    total_loss = all_losses.sum() / norm.sum()

    return total_loss


In [59]:
import pandas as pd
import os
import time
import gc

from tqdm import tqdm
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

# Initialize the stratifier
stratifier = MultilabelStratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [60]:
train_split = []
test_split = []
for fold, (train_patients, test_patients) in enumerate(stratifier.split(y_original_format, y_original_format)):
    print(f"Processing fold {fold}")
    train_split.append(train_patients)
    test_split.append(test_patients)

Processing fold 0
Processing fold 1
Processing fold 2
Processing fold 3
Processing fold 4


In [61]:
def check_organs(organ_series):
    required_organs = {"liver", "spleen", "kidneys", "bowel"}
    return required_organs.issubset(set(organ_series))

In [ ]:
for fold in range(0,5):
    log_dir = 'logs'
    model_dir = 'best_lstm_model'
    n_epochs = 5
    DEBUG = True
    p_mixup = 0.5
    kernel_type ='not_real'
    
    log_file = os.path.join(log_dir, f'{kernel_type}.txt')
    model_file = os.path.join(model_dir, f'{kernel_type}_fold{fold}_best.pth')
    
    train_patients, test_patients = train_split[fold], test_split[fold]#train_patients, test_patients = next(stratifier.split(y_original_format, y_original_format))

    train_df = merged_df[merged_df['patient_id'].isin(train_patients)]

    organ_check = train_df.groupby('series_id')['organ'].agg(check_organs)
    missing_organs_series_ids = organ_check[~organ_check].index.tolist()
    train_df = train_df[~train_df['series_id'].isin(missing_organs_series_ids)]
    
    normal_cases = train_df[(train_df['extravasation_injury']==0) 
                & (train_df['kidney_healthy']==1)
                & (train_df['liver_healthy']==1)
                & (train_df['spleen_healthy']==1)
                & (train_df['bowel_injury']==0)].series_id.unique()
    
    ids_to_remove = random.sample(list(normal_cases), len(train_df.series_id.unique())%4)
    
    train_df = train_df[~train_df['series_id'].isin(ids_to_remove)]
    train_dataset = OrganTrainDataset(train_df, transform=transforms_train)
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=0)
    
    test_df = merged_df[merged_df['patient_id'].isin(test_patients)]
    organ_check = test_df.groupby('series_id')['organ'].agg(check_organs)
    missing_organs_series_ids = organ_check[~organ_check].index.tolist()
    test_df = test_df[~test_df['series_id'].isin(missing_organs_series_ids)]
    
    test_dataset = OrganTrainDataset(test_df, transform=transforms_valid)
    test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=2, shuffle=False, num_workers=0)
    
    metric_best = np.inf
    loss_min = np.inf
    
    model_device = torch.device("cuda:0")
    
    model = TimmModelWithDualLSTM(backbone=backbone, pretrained=True)
    model.to(model_device)
    
    if torch.cuda.device_count() > 1:
        print("Using", torch.cuda.device_count(), "GPUs!")
        model = nn.DataParallel(model)
    
    init_lr = 1e-4
    eta_min = 25e-6
    
    optimizer = optim.AdamW(model.parameters(), lr=init_lr)
    scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, n_epochs, eta_min=eta_min)
    
    use_amp = True
    scaler = torch.cuda.amp.GradScaler() if use_amp else None
    
    
    for epoch in range(1, n_epochs+1):
    # epoch = 1
        model.train()
    
        print(time.ctime(), 'Epoch:', epoch)
    
        train_loss = []
        bowel_binary_loss_list = []
        extravasation_binary_loss_list = []
        kidney_multi_loss_list = []
        liver_multi_loss_list = []
        spleen_multi_loss_list = []
        
        bar = tqdm(train_dataloader)
        
        for images, targets in bar:
            images = images.to(model_device)
            targets = targets.to(model_device)
        
            do_mixup = False
            if random.random() < p_mixup:
                do_mixup = True
                images, targets, targets_mix, lam = mixup(images, targets)
        
            # print(torch.unique(targets))
    
            with amp.autocast():
                logits_binary, logits_multiclass = model(images)
                
                # Compute individual losses
                liver_multi_loss = liver_multi_criterion(logits_multiclass, targets, model_device)
        
                bowel_binary_loss = bowel_binary_criterion(logits_binary, targets, model_device)
                
                extravasation_binary_loss = extravasation_binary_criterion(logits_binary, targets, model_device)
                
                kidney_multi_loss = kidney_multi_criterion(logits_multiclass, targets, model_device)
        
                spleen_multi_loss = spleen_multi_criterion(logits_multiclass, targets, model_device)
            
                # Define your weights
                w_bowel = 1.0
                w_extravasation = 1.0
                w_kidney = 1.0
                w_liver = 1.0
                w_spleen = 1.0
                
                # Compute the weighted combination
                loss = (w_bowel * bowel_binary_loss +
                                 w_extravasation * extravasation_binary_loss +
                                 w_kidney * kidney_multi_loss +
                                 w_liver * liver_multi_loss +
                                 w_spleen * spleen_multi_loss) / (w_bowel + w_extravasation + w_kidney + w_liver + w_spleen)     
                # If mixup is performed, adjust the loss
                if do_mixup:
                    liver_multi_mix = liver_multi_criterion(logits_multiclass, targets_mix, model_device)
        
                    loss = loss * lam + (w_liver * liver_multi_loss) / (w_liver ) * (1 - lam)
        
                    bowel_binary_mix = bowel_binary_criterion(logits_binary, targets_mix, model_device)
                    
                    extravasation_binary_mix = extravasation_binary_criterion(logits_binary, targets_mix, model_device)
                    
                    kidney_multi_mix = kidney_multi_criterion(logits_multiclass, targets_mix, model_device)
                
                    spleen_multi_mix = spleen_multi_criterion(logits_multiclass, targets_mix, model_device)
                    
                    loss = loss * lam + (w_bowel * bowel_binary_loss +
                                             w_extravasation * extravasation_binary_loss +
                                             w_kidney * kidney_multi_loss +
                                             w_liver * liver_multi_loss +
                                             w_spleen * spleen_multi_loss) / (w_bowel + w_extravasation + w_kidney + w_liver + w_spleen) * (1 - lam)
        
            liver_multi_loss_list.append(liver_multi_loss.item()) 
        
            bowel_binary_loss_list.append(bowel_binary_loss.item())
            
            extravasation_binary_loss_list.append(extravasation_binary_loss.item())
            
            kidney_multi_loss_list.append(kidney_multi_loss.item()) 
        
            spleen_multi_loss_list.append(spleen_multi_loss.item())
        
            train_loss.append(loss.item())
            
            # Backward pass and optimization
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            scheduler_cosine.step()
        
            # if train_loss and bowel_binary_loss and extravasation_binary_loss and kidney_multi_loss and liver_multi_loss and spleen_multi_loss:
            bar.set_description(f'Train Loss: {np.mean(train_loss):.4f}')#, Bowel Loss: {np.mean(bowel_binary_loss):.4f}, Ext Loss: {np.mean(extravasation_binary_loss):.4f}, Kidney Loss: {np.mean(kidney_multi_loss):.4f}, Liver Loss: {np.mean(liver_multi_loss):.4f}, Spleen Loss: {np.mean(spleen_multi_loss):.4f}')
    
        model.eval()  # Set the model to evaluation mode
        
        test_loss = []
        bowel_binary_loss_list = []
        extravasation_binary_loss_list = []
        kidney_multi_loss_list = []
        liver_multi_loss_list = []
        spleen_multi_loss_list = []
    
        metric = 0 
        bar = tqdm(test_dataloader)
    
        with torch.no_grad():  # Disable gradient computation during validation
            for images, targets in bar:
                images = images.to(model_device)
                targets = targets.to(model_device)
    
                logits_binary, logits_multiclass = model(images)
                            
                # Compute individual losses
                liver_multi_loss = liver_multi_criterion(logits_multiclass, targets, model_device)
        
                bowel_binary_loss = bowel_binary_criterion(logits_binary, targets, model_device)
                
                extravasation_binary_loss = extravasation_binary_criterion(logits_binary, targets, model_device)
                
                kidney_multi_loss = kidney_multi_criterion(logits_multiclass, targets, model_device)
        
                spleen_multi_loss = spleen_multi_criterion(logits_multiclass, targets, model_device)
            
                # Define your weights
                w_bowel = 1.0
                w_extravasation = 1.0
                w_kidney = 1.0
                w_liver = 1.0
                w_spleen = 1.0
                
                # Compute the weighted combination
                loss = (w_bowel * bowel_binary_loss +
                                 w_extravasation * extravasation_binary_loss +
                                 w_kidney * kidney_multi_loss +
                                 w_liver * liver_multi_loss +
                                 w_spleen * spleen_multi_loss) / (w_bowel + w_extravasation + w_kidney + w_liver + w_spleen)     
               
            liver_multi_loss_list.append(liver_multi_loss.item()) 
        
            bowel_binary_loss_list.append(bowel_binary_loss.item())
            
            extravasation_binary_loss_list.append(extravasation_binary_loss.item())
            
            kidney_multi_loss_list.append(kidney_multi_loss.item()) 
        
            spleen_multi_loss_list.append(spleen_multi_loss.item())
        
            test_loss.append(loss.item())
        
            # if train_loss and bowel_binary_loss and extravasation_binary_loss and kidney_multi_loss and liver_multi_loss and spleen_multi_loss:
            bar.set_description(f'Test Loss: {np.mean(test_loss):.4f}')#, Bowel Loss: {np.mean(bowel_binary_loss):.4f}, Ext Loss: {np.mean(extravasation_binary_loss):.4f}, Kidney Loss: {np.mean(kidney_multi_loss):.4f}, Liver Loss: {np.mean(liver_multi_loss):.4f}, Spleen Loss: {np.mean(spleen_multi_loss):.4f}')
    
        metric = np.mean(test_loss)
        content = f'Fold {fold}, Epoch {epoch}, lr: {optimizer.param_groups[0]["lr"]:.7f}, train loss: {np.mean(train_loss):.5f}, valid loss: {np.mean(test_loss):.5f}, metric: {metric:.6f}.'
        print(content)
    
        if metric < metric_best:
            print(f'metric_best ({metric_best:.6f} --> {metric:.6f}). Saving model ...')
    #             if not DEBUG:
            torch.save(model.state_dict(), model_file)
            metric_best = metric

           # Save Last
        torch.save(
            {
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scaler_state_dict': scaler.state_dict() if scaler else None,
                'score_best': metric_best,
            },
            model_file.replace('_best', '_last')
        )

    del model
    torch.cuda.empty_cache()
    gc.collect()

Using 2 GPUs!
Mon Oct 16 22:12:23 2023 Epoch: 1


  0%|                                                                                     | 0/39 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/torch/optim/lr_scheduler.py:139: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
100%|████████████████████████████████████████████████████████████████████████████| 20/20 [00:21<00:00,  1.09s/it]


Fold 0, Epoch 1, lr: 0.0000928, train loss: 0.33011, valid loss: 0.31598, metric: 0.315980.
metric_best (inf --> 0.315980). Saving model ...
Mon Oct 16 22:14:27 2023 Epoch: 2


100%|████████████████████████████████████████████████████████████████████████████| 20/20 [00:22<00:00,  1.10s/it]


Fold 0, Epoch 2, lr: 0.0000741, train loss: 0.33059, valid loss: 0.32259, metric: 0.322587.
Mon Oct 16 22:16:31 2023 Epoch: 3


...

  0%|                                                                                     | 0/36 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/torch/optim/lr_scheduler.py:139: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
100%|████████████████████████████████████████████████████████████████████████████| 26/26 [00:24<00:00,  1.05it/s]


Fold 1, Epoch 1, lr: 0.0000322, train loss: 0.34626, valid loss: 0.31870, metric: 0.318702.
metric_best (inf --> 0.318702). Saving model ...
Mon Oct 16 22:24:50 2023 Epoch: 2


100%|████████████████████████████████████████████████████████████████████████████| 26/26 [00:25<00:00,  1.02it/s]


Fold 1, Epoch 2, lr: 0.0000741, train loss: 0.34336, valid loss: 0.29303, metric: 0.293033.
metric_best (0.318702 --> 0.293033). Saving model ...
Mon Oct 16 22:26:55 2023 Epoch: 3


Train Loss: 0.3688:  44%|████████████████████████▉                               | 16/36 [00:46<01:07,  3.38s/it]

In [39]:
from sklearn.metrics import classification_report

# Assuming you've defined all your necessary functions, imports, and setup above...

predict_list = []
true_list = []

for fold in range(5):
    
    log_dir = 'logs'
    model_dir = 'best_lstm_model'
    kernel_type ='not_real'
    
    log_file = os.path.join(log_dir, f'{kernel_type}.txt')
    model_file = os.path.join(model_dir, f'{kernel_type}_fold{fold}_best.pth')
    
    train_patients, test_patients = train_split[fold], test_split[fold]#train_patients, test_patients = next(stratifier.split(y_original_format, y_original_format))
    
    test_df = merged_df[merged_df['patient_id'].isin(test_patients)]
    organ_check = test_df.groupby('series_id')['organ'].agg(check_organs)
    missing_organs_series_ids = organ_check[~organ_check].index.tolist()
    test_df = test_df[~test_df['series_id'].isin(missing_organs_series_ids)]
    
    test_dataset = OrganTrainDataset(test_df, transform=transforms_valid)
    test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=2, shuffle=False, num_workers=0)
    
    model_device = torch.device("cuda:0")
    model = TimmModelWithDualLSTM(backbone=backbone, pretrained=True)
    model.to(model_device)
    
    if torch.cuda.device_count() > 1:
        print("Using", torch.cuda.device_count(), "GPUs!")
        model = nn.DataParallel(model)

    # Load the best model for this fold
    model.load_state_dict(torch.load(model_file))
    model.eval()  # Set the model to evaluation mode

    true_labels = {
        'bowel': [],
        'extravasation': [],
        'kidney': [],
        'liver': [],
        'spleen': []
    }

    predicted_labels = {
        'bowel': [],
        'extravasation': [],
        'kidney': [],
        'liver': [],
        'spleen': []
    }

    with torch.no_grad():
        for images, targets in test_dataloader:
            images = images.to(model_device)
            targets = targets.to(model_device)

            print(targets.shape)
            logits_binary, logits_multiclass = model(images)
            
            # This needs adjustment based on your actual tensor shapes and meanings:
            true_labels['bowel'].extend(targets[:, 0].cpu().numpy())
            true_labels['extravasation'].extend(targets[:, 1].cpu().numpy())
            true_labels['kidney'].extend(targets[:, 2:5].cpu().numpy())
            true_labels['liver'].extend(targets[:, 5:8].cpu().numpy())
            true_labels['spleen'].extend(targets[:, 8:11].cpu().numpy())

            # Binary classifications
            predicted_labels['bowel'].extend((torch.sigmoid(logits_binary[:, 0]) >= 0.5).long().cpu().numpy())
            predicted_labels['extravasation'].extend((torch.sigmoid(logits_binary[:, 1]) >= 0.5).long().cpu().numpy())
            
            # Multi-class classifications
            predicted_labels['kidney'].extend(torch.argmax(logits_multiclass[:, :3], dim=1).cpu().numpy())  # Assuming 3 classes for kidney
            predicted_labels['liver'].extend(torch.argmax(logits_multiclass[:, 3:6], dim=1).cpu().numpy())  # Assuming 3 classes for liver
            predicted_labels['spleen'].extend(torch.argmax(logits_multiclass[:, 6:], dim=1).cpu().numpy())  # Assuming 3 classes for spleen

    # print(f"Fold {fold} Results:")
    # for organ in true_labels.keys():
    #     report = classification_report(true_labels[organ], predicted_labels[organ], target_names=[f'Not {organ}', organ])
    #     print(f"Classification Report for {organ}:")
    #     print(report)
    #     print('-' * 50)

    predict_list.append(predicted_labels)
    true_list.append(true_labels)
    
    del model
    torch.cuda.empty_cache()
    gc.collect()

Using 2 GPUs!
torch.Size([2, 11])
torch.Size([2, 11])
torch.Size([2, 11])
torch.Size([2, 11])
torch.Size([2, 11])
torch.Size([2, 11])
torch.Size([2, 11])
torch.Size([2, 11])
torch.Size([2, 11])
torch.Size([2, 11])
torch.Size([2, 11])
torch.Size([2, 11])
torch.Size([2, 11])
torch.Size([2, 11])
torch.Size([2, 11])
torch.Size([2, 11])
torch.Size([2, 11])
torch.Size([2, 11])
torch.Size([2, 11])
torch.Size([1, 11])
Using 2 GPUs!
torch.Size([2, 11])
torch.Size([2, 11])
torch.Size([2, 11])
torch.Size([2, 11])
torch.Size([2, 11])
torch.Size([2, 11])
torch.Size([2, 11])
torch.Size([2, 11])
torch.Size([2, 11])
torch.Size([2, 11])
torch.Size([2, 11])
torch.Size([2, 11])
torch.Size([2, 11])
torch.Size([2, 11])
torch.Size([2, 11])
torch.Size([2, 11])
torch.Size([2, 11])
torch.Size([2, 11])
torch.Size([2, 11])
torch.Size([2, 11])
torch.Size([2, 11])
torch.Size([2, 11])
torch.Size([2, 11])
torch.Size([2, 11])
torch.Size([2, 11])
torch.Size([1, 11])
Using 2 GPUs!
torch.Size([2, 11])
torch.Size([2, 11]

In [43]:
predict_list[0]['spleen']# true_list

[2,
 2,
 0,
 2,
 1,
 0,
 0,
 2,
 2,
 0,
 2,
 0,
 0,
 2,
 2,
 0,
 2,
 2,
 2,
 0,
 2,
 1,
 0,
 0,
 0,
 2,
 0,
 0,
 2,
 0,
 0,
 0,
 1,
 2,
 1,
 0,
 2,
 0,
 0]

In [44]:
true_list[0]['spleen']

[array([1., 0., 0.], dtype=float32),
 array([1., 0., 0.], dtype=float32),
 array([1., 0., 0.], dtype=float32),
 array([1., 0., 0.], dtype=float32),
 array([1., 0., 0.], dtype=float32),
 array([1., 0., 0.], dtype=float32),
 array([1., 0., 0.], dtype=float32),
 array([1., 0., 0.], dtype=float32),
 array([1., 0., 0.], dtype=float32),
 array([1., 0., 0.], dtype=float32),
 array([1., 0., 0.], dtype=float32),
 array([1., 0., 0.], dtype=float32),
 array([1., 0., 0.], dtype=float32),
 array([1., 0., 0.], dtype=float32),
 array([1., 0., 0.], dtype=float32),
 array([1., 0., 0.], dtype=float32),
 array([1., 0., 0.], dtype=float32),
 array([1., 0., 0.], dtype=float32),
 array([1., 0., 0.], dtype=float32),
 array([1., 0., 0.], dtype=float32),
 array([1., 0., 0.], dtype=float32),
 array([1., 0., 0.], dtype=float32),
 array([0., 1., 0.], dtype=float32),
 array([1., 0., 0.], dtype=float32),
 array([1., 0., 0.], dtype=float32),
 array([1., 0., 0.], dtype=float32),
 array([1., 0., 0.], dtype=float32),
 

In [37]:
fold = 0

train_patients, test_patients = train_split[fold], test_split[fold]#train_patients, test_patients = next(stratifier.split(y_original_format, y_original_format))

test_df = merged_df[merged_df['patient_id'].isin(test_patients)]
organ_check = test_df.groupby('series_id')['organ'].agg(check_organs)
missing_organs_series_ids = organ_check[~organ_check].index.tolist()
test_df = test_df[~test_df['series_id'].isin(missing_organs_series_ids)]

In [38]:
test_df

,series_id,organ,image_path,image_name,patient_id,bowel_injury,extravasation_injury,kidney_healthy,kidney_low,kidney_high,liver_healthy,liver_low,liver_high,spleen_healthy,spleen_low,spleen_high
180,9,liver,series_image_split/9/liver/1.jpg,1,403,0,0,1,0,0,1,0,0,1,0,0
181,9,liver,series_image_split/9/liver/4.jpg,4,403,0,0,1,0,0,1,0,0,1,0,0
182,9,liver,series_image_split/9/liver/7.jpg,7,403,0,0,1,0,0,1,0,0,1,0,0
183,9,liver,series_image_split/9/liver/10.jpg,10,403,0,0,1,0,0,1,0,0,1,0,0
184,9,liver,series_image_split/9/liver/13.jpg,13,403,0,0,1,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555091,64423,bowel,series_image_split/64423/bowel/100.jpg,100,2715,0,0,1,0,0,1,0,0,1,0,0
555092,64423,bowel,series_image_split/64423/bowel/103.jpg,103,2715,0,0,1,0,0,1,0,0,1,0,0
555093,64423,bowel,series_image_split/64423/bowel/106.jpg,106,2715,0,0,1,0,0,1,0,0,1,0,0
555094,64423,bowel,series_image_split/64423/bowel/109.jpg,109,2715,0,0,1,0,0,1,0,0,1,0,0


In [ ]:
del model
torch.cuda.empty_cache()
gc.collect()